In [1]:
import quandl
import pandas as pd
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import hypertools as hyp
from datetime import datetime, timedelta

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [6]:
class CurrencyPair:
    
    def __init__(self, pair):
        self.pair = pair
        
    
    def getQuotes(self):
        fw_mar=pd.read_csv('/home/gio/findata/fw/fw_'+self.pair+'_mar',index_col=0, parse_dates=True)
        fw_jun=pd.read_csv('/home/gio/findata/fw/fw_'+self.pair+'_jun',index_col=0, parse_dates=True)
        fw_sep=pd.read_csv('/home/gio/findata/fw/fw_'+self.pair+'_sep',index_col=0, parse_dates=True)
        fw_dec=pd.read_csv('/home/gio/findata/fw/fw_'+self.pair+'_dec',index_col=0, parse_dates=True)
        
        if self.pair == 'gbpusd':
            string = "FED/RXI_US_N_B_UK"
        elif self.pair == 'usdjpy':
            string = "FED/RXI_N_B_JA"
        elif self.pair == 'eurusd':
            string = "FED/RXI_US_N_B_EU"
        spot = quandl.get(string, authtoken="9jpLoG3FHVZqzxGaShTM")
        spot = spot.rename(columns={'Value':self.pair})
        
        libor1m_cur2 = pd.read_csv('/home/gio/findata/libor/'+self.pair[0:3].upper()+'1MTD156N.csv', header=1, index_col=0, names=['libor 1m '+self.pair[0:3]], na_values=['.'], parse_dates=True)
        libor3m_cur2 = pd.read_csv('/home/gio/findata/libor/'+self.pair[0:3].upper()+'3MTD156N.csv', header=1, index_col=0, names=['libor 3m '+self.pair[0:3]], na_values=['.'], parse_dates=True)
        libor1w_cur2 = pd.read_csv('/home/gio/findata/libor/'+self.pair[0:3].upper()+'1WKD156N.csv', header=1, index_col=0, names=['libor 1w '+self.pair[0:3]], na_values=['.'], parse_dates=True)
        
        libor1m_cur1 = pd.read_csv('/home/gio/findata/libor/'+self.pair[3:6].upper()+'1MTD156N.csv', header=1, index_col=0, names=['libor 1m '+self.pair[3:6]], na_values=['.'], parse_dates=True)
        libor3m_cur1 = pd.read_csv('/home/gio/findata/libor/'+self.pair[3:6].upper()+'3MTD156N.csv', header=1, index_col=0, names=['libor 3m '+self.pair[3:6]], na_values=['.'], parse_dates=True)
        libor1w_cur1 = pd.read_csv('/home/gio/findata/libor/'+self.pair[3:6].upper()+'1WKD156N.csv', header=1, index_col=0, names=['libor 1w '+self.pair[3:6]], na_values=['.'], parse_dates=True)
        
        self.df = spot.join([libor1m_cur2, libor1m_cur1,
                         libor3m_cur2, libor3m_cur1,
                         libor1w_cur2, libor1w_cur1],how='outer')
        self.df = self.df.ffill() # fill NA using the preceeding value
        self.df = self.df.dropna(subset=['libor 1m '+self.pair[0:3],'libor 1m '+self.pair[3:6]]) 
        
        self.forwards = [fw_mar,fw_jun,fw_sep,fw_dec]
    
    def apply_cip_dev(self):
        
        for i in self.df.index:
            try:
                cip_dev(i, self.pair, self.forwards, self.df)
            except IndexError, e:
                #print 'IndexError @ date:',i
                continue
            except ValueError, v:
                #print('ValueError %s @ date: %s') %(v,i)
                continue
    
    
    def apply_last_bench(self):
        
        self.df = self.df.dropna(subset=['cip_dev_'+self.pair])
        last_bench(self.pair,self.df)
        
    ##################
    #Machine Learning#
    ##################
    
    def apply_RandomForestReg(self):
        "returns a 2cols df: real and predicted ys"
        RandomForestReg(self.pair, self.df)
        self.predictions = RFpredictions
        self.clf = trainedClf
    
    def apply_GradientBoostingReg(self):
        "returns a 2cols df: real and predicted ys"
        GradientBoostingReg(self.pair, self.df)
        self.predictions = GBpredictions
        self.clf = trainedClf

            
        


In [17]:
gbpusd=CurrencyPair('usdjpy')

In [18]:
gbpusd.getQuotes()

In [19]:
gbpusd.apply_cip_dev()

In [20]:
gbpusd.apply_last_bench()

In [21]:
gbpusd.df.head()

,usdjpy,libor 1m usd,libor 1m jpy,libor 3m usd,libor 3m jpy,libor 1w usd,libor 1w jpy,fw_usdjpy,cip_dev_usdjpy,r_jpy,r_usd,delta_cip_dev_usdjpy,usdjpy_prevDayChange,usdjpy_nextDayChange
1986-01-03,202.53,8.1250,7.2500,8.0625,6.6875,NaN,NaN,202.839757,-0.003156,0.013708,0.015312,0.000000,NaN,-0.003111
1986-01-06,201.90,8.1875,7.1875,8.1250,6.7500,NaN,NaN,201.207243,0.001728,0.013015,0.014769,0.004884,-0.003111,-0.000991
1986-01-07,201.70,8.1250,7.0625,8.1250,6.7500,NaN,NaN,201.005025,0.001658,0.012606,0.014443,-0.000070,-0.000991,0.001983
1986-01-08,202.10,8.0000,6.8750,8.0000,6.6875,NaN,NaN,201.857085,-0.000699,0.012095,0.014014,-0.002357,0.001983,0.000000
1986-01-09,202.10,8.1875,7.0625,8.1875,6.8125,NaN,NaN,201.166767,0.002797,0.012227,0.014113,0.003496,0.000000,0.001237


In [22]:
gbpusd.apply_RandomForestReg()

Fitting 1 folds for each of 17328 candidates, totalling 17328 fits


[Parallel(n_jobs=-1)]: Done 936 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 5736 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 13736 tasks      | elapsed:   57.7s
[Parallel(n_jobs=-1)]: Done 17328 out of 17328 | elapsed:  1.3min finished


Best hyper-parameters are: {'max_features': 'auto', 'min_samples_split': 2, 'splitter': 'random', 'max_depth': 15, 'min_samples_leaf': 2} they give a MSE of: 5.21663382408e-08

MSE for final classifier on test is 1.57594059034e-07


In [23]:
gbpusd.predictions.head()

,usdjpy_nextDayChange,RFpred_usdjpy_nextDayChange
0,0.002920,0.002625
1,-0.004659,-0.004901
2,-0.011535,-0.011800
3,0.010654,0.010839
4,-0.000084,0.000177


## FUNCTIONS

In [15]:
def cip_dev(date, pair, forwards, df):
    "in a gbpusd example cur1 is usd..."    
    cur1 = pair[3:6]
    cur2 = pair[0:3]
    fw_mar = forwards[0]
    fw_jun = forwards[1]
    fw_sep = forwards[2]
    fw_dec = forwards[3]
    
    global r1
    global r2
    global forward
    # if date is before 15th of January
    if datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('01-15',"%m-%d"):
        maturity = str(date)[0:4]+'-03-15'
        if date in fw_mar.index:
            forward = fw_mar[fw_mar.index == date]['fw'][0]
        else:
            forward = fw_mar[fw_mar.index == (date+timedelta(1))]['fw'][0]
        first_term = str(date+timedelta(60))[0:10]
        if pd.isnull(df[df.index == date]['libor 1w '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 1w '+cur2])[0] == False:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(2/12))*(1 + df[df.index == date ]['libor 1w '+cur1][0]/100)**(days_between(first_term,maturity)/365)
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(2/12))*(1 + df[df.index == date ]['libor 1w '+cur2][0]/100)**(days_between(first_term,maturity)/365)
        else:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(str(date)[0:10],maturity)/365))
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(str(date)[0:10],maturity)/365))
    # if date is before 15th of February
    elif datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('02-15',"%m-%d"):
        maturity = str(date)[0:4]+'-03-15'
        if date in fw_mar.index:
            forward = fw_mar[fw_mar.index == date]['fw'][0]
        else:
            forward = fw_mar[fw_mar.index == (date+timedelta(1))]['fw'][0]
        first_term = str(date+timedelta(30))[0:10]
        if pd.isnull(df[df.index == date]['libor 1w '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 1w '+cur2])[0] == False:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(1/12))*(1 + df[df.index == date ]['libor 1w '+cur1][0]/100)**(days_between(first_term,maturity)/365)
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(1/12))*(1 + df[df.index == date ]['libor 1w '+cur2][0]/100)**(days_between(first_term,maturity)/365)
        else:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(str(date)[0:10],maturity)/365))
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(str(date)[0:10],maturity)/365))
    #  March 15th
    elif datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('03-15',"%m-%d"):
        maturity = str(date)[0:4]+'-06-15'
        if date in fw_jun.index:
            forward = fw_jun[fw_jun.index == date]['fw'][0]
        else:
            forward = fw_jun[fw_jun.index == (date+timedelta(1))]['fw'][0]
        if pd.isnull(df[df.index == date]['libor 3m '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 3m ' + cur2])[0] == False:
            r1 = (1 + df[df.index == date ]['libor 3m '+cur1][0]/100)**(days_between(maturity,str(date)[0:10])/365)
            r2 = (1 + df[df.index == date ]['libor 3m ' + cur2][0]/100)**(days_between(maturity,str(date)[0:10])/365)
        else:
            r1 = (1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(maturity,str(date)[0:10])/365)
            r2 = (1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(maturity,str(date)[0:10])/365)
    # April 15th
    elif datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('04-15',"%m-%d"):
        maturity = str(date)[0:4]+'-06-15'
        if date in fw_jun.index:
            forward = fw_jun[fw_jun.index == date]['fw'][0]
        else:
            forward = fw_jun[fw_jun.index == (date+timedelta(1))]['fw'][0]
        first_term = str(date+timedelta(60))[0:10]
        if pd.isnull(df[df.index == date]['libor 1w '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 1w '+cur2])[0] == False:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(2/12))*(1 + df[df.index == date ]['libor 1w '+cur1][0]/100)**(days_between(first_term,maturity)/365)
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(2/12))*(1 + df[df.index == date ]['libor 1w '+cur2][0]/100)**(days_between(first_term,maturity)/365)
        else:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(str(date)[0:10],maturity)/365))
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(str(date)[0:10],maturity)/365))
    # May 15th
    elif datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('05-15',"%m-%d"):
        maturity = str(date)[0:4]+'-06-15'
        if date in fw_jun.index:
            forward = fw_jun[fw_jun.index == date]['fw'][0]
        else:
            forward = fw_jun[fw_jun.index == (date+timedelta(1))]['fw'][0]
        first_term = str(date+timedelta(30))[0:10]
        if pd.isnull(df[df.index == date]['libor 1w '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 1w '+cur2])[0] == False:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(1/12))*(1 + df[df.index == date ]['libor 1w '+cur1][0]/100)**(days_between(first_term,maturity)/365)
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(1/12))*(1 + df[df.index == date ]['libor 1w '+cur2][0]/100)**(days_between(first_term,maturity)/365)
        else:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(str(date)[0:10],maturity)/365))
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(str(date)[0:10],maturity)/365))
    # June 15th
    elif datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('06-15',"%m-%d"):
        maturity = str(date)[0:4]+'-09-15'
        if date in fw_sep.index:
            forward = fw_sep[fw_sep.index == date]['fw'][0]
        else:
            forward = fw_sep[fw_sep.index == (date+timedelta(1))]['fw'][0]
        if pd.isnull(df[df.index == date]['libor 3m '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 3m ' + cur2])[0] == False:
            r1 = (1 + df[df.index == date ]['libor 3m '+cur1][0]/100)**(days_between(maturity,str(date)[0:10])/365)
            r2 = (1 + df[df.index == date ]['libor 3m ' + cur2][0]/100)**(days_between(maturity,str(date)[0:10])/365)
        else:
            r1 = (1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(maturity,str(date)[0:10])/365)
            r2 = (1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(maturity,str(date)[0:10])/365)
    #July 15th
    elif datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('07-15',"%m-%d"):
        maturity = str(date)[0:4]+'-09-15'
        if date in fw_sep.index:
            forward = fw_sep[fw_sep.index == date]['fw'][0]
        else:
            forward = fw_sep[fw_sep.index == (date+timedelta(1))]['fw'][0]
        first_term = str(date+timedelta(60))[0:10]
        if pd.isnull(df[df.index == date]['libor 1w '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 1w '+cur2])[0] == False:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(2/12))*(1 + df[df.index == date ]['libor 1w '+cur1][0]/100)**(days_between(first_term,maturity)/365)
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(2/12))*(1 + df[df.index == date ]['libor 1w '+cur2][0]/100)**(days_between(first_term,maturity)/365)
        else:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(str(date)[0:10],maturity)/365))
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(str(date)[0:10],maturity)/365))
    # August 15th
    elif datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('08-15',"%m-%d"):
        maturity = str(date)[0:4]+'-09-15'
        if date in fw_sep.index:
            forward = fw_sep[fw_sep.index == date]['fw'][0]
        else:
            forward = fw_sep[fw_sep.index == (date+timedelta(1))]['fw'][0]
        first_term = str(date+timedelta(30))[0:10]
        if pd.isnull(df[df.index == date]['libor 1w '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 1w '+cur2])[0] == False:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(1/12))*(1 + df[df.index == date ]['libor 1w '+cur1][0]/100)**(days_between(first_term,maturity)/365)
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(1/12))*(1 + df[df.index == date ]['libor 1w '+cur2][0]/100)**(days_between(first_term,maturity)/365)
        else:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(str(date)[0:10],maturity)/365))
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(str(date)[0:10],maturity)/365))
    # September 15th
    elif datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('09-15',"%m-%d"):
        maturity = str(date)[0:4]+'-12-15'
        if date in fw_dec.index:
            forward = fw_dec[fw_dec.index == date]['fw'][0]
        else:
            forward = fw_dec[fw_dec.index == (date+timedelta(1))]['fw'][0]
        if pd.isnull(df[df.index == date]['libor 3m '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 3m ' + cur2])[0] == False:
            r1 = (1 + df[df.index == date ]['libor 3m '+cur1][0]/100)**(days_between(maturity,str(date)[0:10])/365)
            r2 = (1 + df[df.index == date ]['libor 3m ' + cur2][0]/100)**(days_between(maturity,str(date)[0:10])/365)
        else:
            r1 = (1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(maturity,str(date)[0:10])/365)
            r2 = (1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(maturity,str(date)[0:10])/365)
    # October 15th
    elif datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('10-15',"%m-%d"):
        maturity = str(date)[0:4]+'-12-15'
        if date in fw_dec.index:
            forward = fw_dec[fw_dec.index == date]['fw'][0]
        else:
            forward = fw_dec[fw_dec.index == (date+timedelta(1))]['fw'][0]
        first_term = str(date+timedelta(60))[0:10]
        if pd.isnull(df[df.index == date]['libor 1w '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 1w '+cur2])[0] == False:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(2/12))*(1 + df[df.index == date ]['libor 1w '+cur1][0]/100)**(days_between(first_term,maturity)/365)
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(2/12))*(1 + df[df.index == date ]['libor 1w '+cur2][0]/100)**(days_between(first_term,maturity)/365)
        else:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(str(date)[0:10],maturity)/365))
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(str(date)[0:10],maturity)/365))
    # November 15th
    elif datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('11-15',"%m-%d"):
        maturity = str(date)[0:4]+'-12-15'
        if date in fw_dec.index:
            forward = fw_dec[fw_dec.index == date]['fw'][0]
        else:
            forward = fw_dec[fw_dec.index == (date+timedelta(1))]['fw'][0]
        first_term = str(date+timedelta(30))[0:10]
        if pd.isnull(df[df.index == date]['libor 1w '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 1w '+cur2])[0] == False:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(1/12))*(1 + df[df.index == date ]['libor 1w '+cur1][0]/100)**(days_between(first_term,maturity)/365)
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(1/12))*(1 + df[df.index == date ]['libor 1w '+cur2][0]/100)**(days_between(first_term,maturity)/365)
        else:
            r1 = ((1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(str(date)[0:10],maturity)/365))
            r2 = ((1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(str(date)[0:10],maturity)/365))
    # December 15th
    elif datetime.strptime(str(date)[5:10],"%m-%d") < datetime.strptime('12-15',"%m-%d"):
        maturity = str(date)[0:4]+'-03-15'
        if date in fw_mar.index:
            forward = fw_mar[fw_mar.index == date]['fw'][0]
        else:
            forward = fw_mar[fw_mar.index == (date+timedelta(1))]['fw'][0]
        if pd.isnull(df[df.index == date]['libor 3m '+cur1])[0] == False and pd.isnull(df[df.index == date]['libor 3m ' + cur2])[0] == False:
            r1 = (1 + df[df.index == date ]['libor 3m '+cur1][0]/100)**(days_between(maturity,str(date)[0:10])/365)
            r2 = (1 + df[df.index == date ]['libor 3m ' + cur2][0]/100)**(days_between(maturity,str(date)[0:10])/365)
        else:
            r1 = (1 + df[df.index == date ]['libor 1m '+cur1][0]/100)**(days_between(maturity,str(date)[0:10])/365)
            r2 = (1 + df[df.index == date ]['libor 1m '+cur2][0]/100)**(days_between(maturity,str(date)[0:10])/365)
    
    spot = df[df.index == date][pair][0]
    cip_deviation = (r1 - r2*forward/spot)
    df.set_value(date, 'fw_'+pair, forward)
    df.set_value(date, 'cip_dev_'+ pair, cip_deviation)
    df.set_value(date, 'r_'+cur1, (r1-1))
    df.set_value(date, 'r_'+cur2, (r2-1))
    #{'fw_'+pair : forward, 'cip_dev_'+pair : cip_deviation, 'r_'+cur1 : (r1-1), 'r_'+cur2 : (r2-1)}
             
    return None 


In [16]:
def last_bench(pair,df):
    "pair must be a string, df is the pandasdf where data for that pair are stored"
    # I define an aux with the first differences, appending to df I must slide down a row because np.diff_i = x_(i+1)-x
    # --> use np.insert to put a zero a zero as first element of the returned np.array 
    aux = np.insert(np.diff(df['cip_dev_'+pair]),0,0)
    # I append it to df as a new column
    df.ix[:,'delta_cip_dev_'+pair]=aux
    
    # spot change FROM PREVIOUS DAY
    for i in range(1,len(df.index)):
        df.set_value(df.index[i],pair+'_prevDayChange',
                     (df.ix[df.index[i],pair]-df.ix[df.index[i-1],pair])/df.ix[df.index[i-1],pair])
    # spot change THE FOLLOWING DAY -> it will be my Y
    for i in range(0,len(df.index)-1):
        df.set_value(df.index[i],pair+'_nextDayChange',
                     (df.ix[df.index[i+1],pair]-df.ix[df.index[i],pair])/df.ix[df.index[i],pair])

    return None

### ML FUNCTIONS

In [5]:
from sklearn import tree
from sklearn import metrics
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import randint as sp_randint

def mse(yval_pred, yval):
  val_mse = metrics.mean_squared_error(yval_pred, yval)
  return(val_mse)

def RandomForestReg(pair, df):
    np.random.seed(0) #important
    
    df=df.dropna()
    
    indicesTrain = np.where(df.index <= '2012')[0]
    indicesVal = np.where((df.index > '2013') & (df.index < '2015'))[0]
    indicesTest = np.where(df.index >= '2015')[0]
    indicesTrainFin = np.where(df.index < '2015')[0]

    partitions = [(indicesTrain, indicesVal)]

    param_grid = {'splitter' : ('best', 'random'),
                 'max_features' : ('log2', 'sqrt', 'auto'),
                 'max_depth' : range(1,20,1),
                 'min_samples_split' : range(2,10,1),
                 'min_samples_leaf' : range(1,20,1)
                 }

    grid_clf = GridSearchCV(tree.DecisionTreeRegressor(), 
                       param_grid = param_grid,
                       scoring = 'neg_mean_squared_error',
                       cv = partitions,
                       verbose = 1, n_jobs = -1)
    
    # naturally pair_change column is omitted
    grid_clf.fit(df.ix[:, df.columns != pair+'_nextDaychange'].as_matrix(), df.ix[:,pair+'_nextDayChange'])

    print "Best hyper-parameters are: {} they give a MSE of: {}".format(grid_clf.best_params_, -grid_clf.best_score_)

    clf = tree.DecisionTreeRegressor(max_features= grid_clf.best_params_['max_features'],
                                     min_samples_split=grid_clf.best_params_['min_samples_split'], 
                                     splitter=grid_clf.best_params_['splitter'],
                                     max_depth=grid_clf.best_params_['max_depth'], 
                                     min_samples_leaf=grid_clf.best_params_['min_samples_leaf'])

    x = df.ix[:, df.columns != pair+'_nextDaychange'].as_matrix()
    y = df[pair+'_nextDayChange'].as_matrix() # slipped next day spot

    xTrain = x[indicesTrainFin]
    yTrain = y[indicesTrainFin]
    xTest = x[indicesTest]
    yTest = y[indicesTest]

    clf.fit(xTrain,yTrain)

    yPred = clf.predict(xTest)

    print "\nMSE for final classifier on test is %s" %(mse(yPred, yTest))
    
    global trainedClf
    trainedClf= clf
    global RFpredictions
    RFpredictions = pd.concat([pd.DataFrame(yTest, columns=[pair+'_nextDayChange']), pd.DataFrame(yPred, columns=['RFpred_'+pair+'_nextDayChange'])], axis=1)

############################################################################################################
    
def GradientBoostingReg(pair, df):
    
    df=df.dropna()
    
    np.random.seed(0)

    indicesTrain = np.where(df.index <= '2012')[0]
    indicesVal = np.where((df.index > '2013') & (df.index < '2015'))[0]
    indicesTest = np.where(df.index >= '2015')[0]
    indicesTrainFin = np.where(df.index < '2015')[0]
    
    partitions = [(indicesTrain, indicesVal)]

    param_dist = {'n_estimators' : sp_randint(25,125)}

    n_iter_search = 150 #Number of parameter settings that are sampled. n_iter trades off runtime vs quality of the solution.

    rs_gb = RandomizedSearchCV(GradientBoostingRegressor(), 
                                       param_distributions=param_dist,
                                       scoring='neg_mean_squared_error',
                                       cv=partitions ,
                                       n_jobs=-1, verbose=1,
                                       n_iter=n_iter_search)

    rs_gb.fit(df.ix[:, df.columns != pair+'_nextDaychange'].as_matrix(), df.ix[:, pair+'_nextDayChange'])

    print "Best hyper-parameters are: {} they give a MAE of: {}".format(rs_gb.best_params_, -rs_gb.best_score_)
    
    clf = GradientBoostingRegressor(n_estimators=rs_gb.best_params_['n_estimators'])

    x = df.ix[:, df.columns != pair+'_nextDayChange'].as_matrix()
    y = df[pair+'_nextDayChange'].as_matrix()

    xTrain = x[indicesTrainFin]
    yTrain = y[indicesTrainFin]
    xTest = x[indicesTest]
    yTest = y[indicesTest]

    clf.fit(xTrain,yTrain)

    yPred = clf.predict(xTest)

    print "\nMSE for final classifier on test is %s" %(mse(yPred, yTest))
    
    global trainedClf
    trainedClf= clf
    global GBpredictions
    GBpredictions = pd.concat([pd.DataFrame(yTest, columns=[pair+'_nextDayChange']), pd.DataFrame(yPred, columns=['GBpred_'+pair+'_nextDayChange'])], axis=1)


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
